In [2]:
from sympy import *

In [3]:
#definição de variáveis
# DV = (delta)V
Vcc, s, L, Iof, I, R, Vdo, Ioa, Vra, Vfr, t, k, T, V0, V1, DV = symbols('Vcc s L Iof I R Vdo Ioa Vra Vfr t k T V0 V1 DV')

In [4]:
# chave aberta
Eq1=Eq(Vcc/s -s*L*I + L*Iof - R*I,0)
Eq1

Eq(-I*L*s - I*R + Iof*L + Vcc/s, 0)

In [5]:
# chave fechada
Eq2=Eq(-Vdo/s-s*L*I+Ioa*L - R*I,0)
Eq2

Eq(-I*L*s - I*R + Ioa*L - Vdo/s, 0)

In [6]:
# define a corrente no circuito com chave fechada
sol1 = solve(Eq1,I)
sol1

[(Iof*L*s + Vcc)/(s*(L*s + R))]

In [7]:
# define a tensão no resistor no circuito com chave fechada
Vrf=sol1[0]*R
Vrf

R*(Iof*L*s + Vcc)/(s*(L*s + R))

In [8]:
# define a corrente no circuito com chave aberta
sol2 = solve(Eq2,I)
sol2

[(Ioa*L*s - Vdo)/(s*(L*s + R))]

In [9]:
# define a tensão no resistor no circuito com chave aberta
Vra=sol2[0]*R
Vra

R*(Ioa*L*s - Vdo)/(s*(L*s + R))

## Análise no Tempo

In [10]:
# Aplicaremos a transformada inversa de Laplace para obter a tensão no domínio do tempo
# Dividimos por Heaviside(t) para elimitar o Euler como função de teta

Vrft = simplify(inverse_laplace_transform(Vrf, s, t))/Heaviside(t)
Vrft

(Iof*R + Vcc*exp(R*t/L) - Vcc)*exp(-R*t/L)

In [11]:
# Aplicaremos a transformada inversa de Laplace para obter a tensão no domínio do tempo
# Dividimos por Heaviside(t) para elimitar o Euler como função de teta

Vrat = simplify(inverse_laplace_transform(Vra, s, t))/Heaviside(t)
Vrat

(Ioa*R - Vdo*exp(R*t/L) + Vdo)*exp(-R*t/L)

## V1 e V2

In [12]:
# substituindo os valores de em t e na corrente inicial (condição inicial) para definir V1
V1=Vrft.subs(t, k*T)
V1=V1.subs(Iof, V0/R)
V1

(V0 + Vcc*exp(R*T*k/L) - Vcc)*exp(-R*T*k/L)

In [13]:
# substituindo os valores de em t e na corrente inicial (condição inicial) para definir V2

V2=Vrat.subs(t, (1-k)*T)
V2=V2.subs(Ioa, V1/R)
V2

(-Vdo*exp(R*T*(1 - k)/L) + Vdo + (V0 + Vcc*exp(R*T*k/L) - Vcc)*exp(-R*T*k/L))*exp(-R*T*(1 - k)/L)

## Análise em regime permanente

In [14]:
# Achar os valores de V0 e DV
# Considerando V2 = V0 e V1 = V0 + DV

# V0 + DV = V1
prop1 = Eq(V0 + DV, V1)

# V0 = V2
prop2 = Eq(V0, V2)

# solucionando o sistema e obtendo os valores de V0 e DV
Vsol = linsolve([prop1, prop2], (V0, DV))

# atribuindo as soluções de V0 e DV
solV0 = Vsol.args[0][0]
solDV = Vsol.args[0][1]
solV0

Vcc*exp(R*T*k/L)/(exp(R*T/L) - 1) - Vcc/(exp(R*T/L) - 1) - Vdo*exp(R*T/L)/(exp(R*T/L) - 1) + Vdo*exp(R*T*k/L)/(exp(R*T/L) - 1)

In [15]:
solDV

Vcc*exp(R*T/L)*exp(R*T*k/L)/(exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) - Vcc*exp(R*T/L)/(exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) - Vcc*exp(2*R*T*k/L)/(exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) + Vcc*exp(R*T*k/L)/(exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) + Vdo*exp(R*T/L)*exp(R*T*k/L)/(exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) - Vdo*exp(R*T/L)/(exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) - Vdo*exp(2*R*T*k/L)/(exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) + Vdo*exp(R*T*k/L)/(exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L))

## Série de Taylor e limites

In [16]:
# aplicando a série de Taylor e os limites DV = Vsol.args[0][1]; V0 = Vsol.args[0][0]
# usa-se o método .removeO() para retirar o termo final da série e obter um polinômio conveniente para a substituição a seguir
tayDV = series(solDV, T, 0, 2).removeO()
tayV0 = series(solV0, T, 0, 2).removeO()
tayDV

T*(-R*Vcc*k**2/L + R*Vcc*k/L - R*Vdo*k**2/L + R*Vdo*k/L)

In [17]:
tayV0

T*(R*Vcc*k**2/(2*L) - R*Vcc*k/(2*L) + R*Vdo*k**2/(2*L) - R*Vdo*k/(2*L)) + Vcc*k + Vdo*k - Vdo

In [20]:
# limites
limTayDV = limit(solDV, T, 0, dir="+-")
limTayV0 = limit(solV0, T, 0, dir="+-")
limTayV0

Vcc*k + Vdo*k - Vdo

In [21]:
limTayDV

0

# Valores para o indutor

In [25]:
Lprocurado = Eq(tayDV.subs(Vdo,0), Vcc/1000)
Lprocurado = solve(Lprocurado, [L])[0]
Eq(L,Lprocurado)

Eq(L, 1000*R*T*k*(1 - k))

# Substituindo os valores

In [ ]:

#R = 5 ohm
#Feq = 10 kHz
L1=Lprocurado.subs([(k, 1/2), (R,5), (T, 1/10000)])

#Feq = 50 kHz
L2=Lprocurado.subs([(k, 1/2), (R,5), (T, 1/50000)])
L2

0.0250000000000000

In [ ]:
#R = 1 ohm
#Feq = 10 kHz
L3=Lprocurado.subs([(k, 1/2), (R,1), (T, 1/10000)])

#Feq = 50 kHz
L4=Lprocurado.subs([(k, 1/2), (R,1), (T, 1/50000)])
L3,L4

(0.0250000000000000, 0.00500000000000000)

In [ ]:
# Imprimindo os resultados
print("Resultados para as indutâncias calculadas:")
print(f"L1 (R = 5 ohm, Feq = 10 kHz): {L1.evalf()}")
print(f"L2 (R = 5 ohm, Feq = 50 kHz): {L2.evalf()}")
print(f"L3 (R = 1 ohm, Feq = 10 kHz): {L3.evalf()}")
print(f"L4 (R = 1 ohm, Feq = 50 kHz): {L4.evalf()}")

Resultados para as indutâncias calculadas:
L1 (R = 5 ohm, Feq = 10 kHz): 0.125000000000000
L2 (R = 5 ohm, Feq = 50 kHz): 0.0250000000000000
L3 (R = 1 ohm, Feq = 10 kHz): 0.0250000000000000
L4 (R = 1 ohm, Feq = 50 kHz): 0.00500000000000000


In [ ]:
import sympy as sp

print("V0 a 50khz")
sp.Eq(V0,Vsol.args[0][0].subs([(k, 1/2), (R,5), (T, 1/50000),(Vcc, 5),(L,L2),(Vdo,0)])),sp.Eq(V0,Vsol.args[0][0].subs([(k, 1/2), (R,5), (T, 1/50000),(Vcc, 5),(L,L1),(Vdo,0)]))


V0 a 50khz


(Eq(V0, 2.49750000083327), Eq(V0, 2.49950000000717))

In [ ]:
sp.Eq(V0,limTayDV.subs([(k, 1/2), (R,5), (T, 1/50000),(Vcc, 5),(L,L3),(Vdo,1)]))

Eq(V0, 0)

In [ ]:
import pandas as pd

parametros = [
    (5, 1/10000, L1),  # R = 5 ohm, Feq = 10 kHz, L = L1
    (5, 1/50000, L2),  
    (1, 1/10000, L3),  
    (1, 1/50000, L4)   
]

# Calculando V0 e DV para cada conjunto de parâmetros
resultados = []
for R_val, T_val, L_val in parametros:
    resultado_V0 = Vsol.args[0][0].subs([(k, 1/2), (R, R_val), (T, T_val), (Vcc, 5), (L, L_val), (Vdo, 0)])
    resultado_DV = Vsol.args[0][1].subs([(k, 1/2), (R, R_val), (T, T_val), (Vcc, 5), (L, L_val), (Vdo, 0)])
    resultados.append((R_val, T_val**(-1), L_val, resultado_V0, resultado_DV))

# Criando um DataFrame a partir da lista de resultados
df = pd.DataFrame(resultados, columns=['R (ohm)', 'Feq (Hz)', 'L (H)', 'V0', 'DV'])

# Exibindo a tabela
print(df)

   R (ohm)  Feq (Hz)                L (H)                V0  \
0        5   10000.0    0.125000000000000  2.49750000083327   
1        5   50000.0   0.0250000000000000  2.49750000083327   
2        1   10000.0   0.0250000000000000  2.49750000083327   
3        1   50000.0  0.00500000000000000  2.49750000083327   

                    DV  
0  0.00499999833300535  
1  0.00499999833300535  
2  0.00499999833300535  
3  0.00499999833300535  
